In [ ]:
import pandas as pd
import numpy as np
from upsetjs_jupyter_widget import UpSetWidget

# Data
### Input File Format
The input file format is similar to "Options 1: File" in UpSetR-shiny (https://github.com/hms-dbmi/UpSetR-shiny)

- Index are the unique ids
- Columns are `set 1, set2, ..., set M` where `set` columns contain either `1` or `0`, `1` indicating the '⬤' representation in UpSet
- Attributes -> can be passed manually

#### Demo Data: COVID Symptom Tracker April 7 (via https://www.nature.com/articles/d41586-020-00154-w)

<img src="https://media.nature.com/lw800/magazine-assets/d41586-020-00154-w/d41586-020-00154-w_17880786.jpg" alt="demo_diagram" style="width:300px;"/>

In [ ]:
df = pd.read_csv("../data/covid_symptoms_table.csv", index_col=0)

df

# Visualization
#### The UpSet.js visualizations contain three main views: 

(1) **vertical bar chart** on the top showing the cardinality of each intersecting set;

(2) **matrix view** on the bottom-right showing the intersecting set;

(3) **horizontal bar chart** on the bottom-left showing the cardinality of each set.

#### Options:

see https://upset.js.org/integrations/jupyter/ for a tutorial and https://upset.js.org/api/jupyter/ for the Python API

1. Specify sets of interest (e.g., `["Comedy", "Action", "Adventure"]`)
 -> subset the DataFrame before passing to UpSet.js

2. Show empty intersections or not
 -> use `generate_intersections(empty=True)` 
 
 see https://upset.js.org/api/jupyter/api.html#upsetjs_jupyter_widget.UpSetWidget.generate_intersections

3. Sorting type: `Frequency` or `Degree`
 - by frequency i.e., set cardinality: `generate_intersections(order.by='cardinality')`
 - by degree: `generate_intersections(order.by='degree')`
 - by set group i.e., first all that have set A then remaining having set B, ..: `generate_intersections(order.by='group')` 
 - by a combination of them `generate_intersections(order.by=('group', 'cardinality', 'name'))`
 
 see https://upset.js.org/api/jupyter/api.html#upsetjs_jupyter_widget.UpSetWidget.generate_intersections

4. Sorting order: `descending` or `ascending`
 
 -> so far fixed per attribute: 
  - ascencding: name,degree,group
  - descending: cardinality
  
  Alternatively, one can sort the sets and combinations manually, e.g.
  ```python
  w.sets = sorted(w.sets, key=lambda s: s.name, reverse=True)
  w.combinations = sorted(w.combinations, key=lambda s: s.name, reverse=True)
  ```

## Examples w/ Different Options

In [ ]:
# UpSetAltair(
#     data=df.copy(), 
#     sets=["Shortness of Breath", "Diarrhea", "Fever", "Cough", "Anosmia", "Fatigue"],
#     abbre=["B", "D", "Fe", "C", "A", "Fa"],
#     sort_by="Frequency",
#     sort_order="ascending"
# )

UpSetWidget().from_dataframe(df)

In [ ]:
# UpSetAltair(
#     data=df.copy(), 
#     sets=["Shortness of Breath", "Diarrhea", "Fever", "Cough", "Anosmia", "Fatigue"],
#     abbre=["B", "D", "Fe", "C", "A", "Fa"],
#     sort_by="Degree",
#     sort_order="ascending"
# )
UpSetWidget().from_dataframe(df).generate_intersections(order_by='degree')

In [ ]:
# UpSetAltair(
#     data=df.copy(), 
#     sets=["Shortness of Breath", "Diarrhea", "Fever", "Cough", "Anosmia", "Fatigue"],
#     abbre=["B", "D", "Fe", "C", "A", "Fa"],
#     sort_by="Frequency",
#     sort_order="ascending",
#     width=1200,
#     height=500,
#     height_ratio=0.6,
#     horizontal_bar_chart_width=300
# )

w = UpSetWidget().from_dataframe(df)
w.width = 1200
w.height = 500
w.height_ratios = (0.6, 0.4)
w.width_ratios = (0.25, 0.15, 0.6)
w

## Attributes

UpSet.js supports numerical attributes that are shown as boxplots

In [ ]:
df_attr = df.copy()
df_attr["Attr"] = np.random.randint(0, 100, size=len(df))

UpSetWidget().from_dataframe(df_attr, attributes=['Attr'])

## Interactivity

UpSet.js is compatible with ipywidgets `interact` concept for interactive widgets

In [ ]:
from ipywidgets import interact

w = UpSetWidget()
w.description = '' # for compatibility with ipywidgets 7.5.1 see https://github.com/upsetjs/upsetjs_jupyter_widget/issues/4
w.from_dataframe(df)

def selection_changed(s):
    return s.name if s else None
interact(selection_changed, s=w)

In [ ]:
w_click  = UpSetWidget()
w_click.description = '' # for compatibility with ipywidgets 7.5.1 see https://github.com/upsetjs/upsetjs_jupyter_widget/issues/4
w_click.from_dataframe(df)
w_click.mode = 'click' # trigger on mouse clicks

def selection_changed(s):
    return s.name if s else None
interact(selection_changed, s=w_click)

In [ ]:
# access current selection
w_click.selection